# Metro Boston Housing Analysis: 2013-2021

Metro Boston housing prices and days on market for the period 2013-2021. In this notebook, we explore geospatial  patterns. We'll create maps of the housing metrics (choropleths). 

Geospatial data contains polygons of Massachusetts municipalities in a geographic coordinate system (WGS84). These data are from [Koordinates](https://koordinates.com).  

NCH 2021

In [ ]:
import json
import geopandas as gpd
import pandas as pd
import plotly.express as px

## Configure

In [ ]:
export_geojson = False #Enable the first time you run the notebook
export_figure  = True  #Write choropleths as png
export_html    = False #Write choropleths as interactive html 

## Helper

In [ ]:
def plot_map(df, geopoly, metric, legend=None, lb=None, ub=None):
    """Plot choropleth map of metro area metric"""
    if lb is None or ub is None:
        lb = data[metric].quantile(.1)
        ub = data[metric].quantile(.9)

    if not legend:
        legend = metric

    fig = px.choropleth(df, geojson=geopoly, locations='OBJECTID', color=metric,
        color_continuous_scale="Viridis", featureidkey="properties.OBJECTID",
        range_color=(lb, ub),
        hover_data=["City/Town"],
        labels={metric:legend}
    )
    
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

    return fig

## Import and format geospatial

Read a municipal (muni) polygons from shapefile and convert to a geojson. 

In [ ]:
path_to_data = './data/kx-massachusetts-municipalities-SHP/massachusetts-municipalities.shp'
gdf = gpd.read_file( path_to_data )

In [ ]:
gdf.head()

These data contain a property with the names of the town/munis. We'll use this to join with the housing dataset after converting both to a lowercase. 

In [ ]:
gdf["TOWN"] = gdf["TOWN"].str.lower()

Convert to a geojson format. 

In [ ]:
filename = "./data/ma-town-geospatial.json"

In [ ]:
if export_geojson:
    df.to_file(filename, driver = "GeoJSON")

Import geojson

In [ ]:
with open(filename) as geofile:
    towns = json.load(geofile)

towns["features"][0]

## Import housing metrics

In [ ]:
df = pd.read_csv( "./data/housingData2020-prepared.csv" )
df['City/Town'] = df['City/Town'].str.lower()

In [ ]:
df.head()

Merge geospatial and housing data to have a  common key, which is `OBJECTID`.

In [ ]:
data = df.merge(gdf[['TOWN', "OBJECTID"]], left_on="City/Town", right_on="TOWN")
data.head()

## Create map

In [ ]:
lb = data["2010 Median Price"].quantile(.1)
ub = data["2020 Median Price"].quantile(.9)

In [ ]:
fig = px.choropleth(data, geojson=towns, locations='OBJECTID', color='2010 Median Price',
     color_continuous_scale="Viridis", featureidkey="properties.OBJECTID",
     range_color=(lb, ub),
     hover_data=["City/Town"],
     labels={'2010 Median Price':'2010 Median Price'}
    )

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

if export_html:
    fig.write_html("./html/2010-boston-housing-median.html")
if export_figure:
    fig.write_image("./figures/boston-housing-2010.png", scale=5)

In [ ]:
fig = plot_map(data, towns, '2019 Median Price', lb=lb, ub=ub )

if export_html:
    fig.write_html("./html/2019-boston-housing-median.html")

In [ ]:
fig = plot_map(data, towns, '2020 Median Price', lb=lb, ub=ub )

if export_html:
    fig.write_html("./html/2020-boston-housing-median.html")
if export_figure:
    fig.write_image("./figures/boston-housing-2020.png", scale=5)

## Change

In [ ]:
fig = plot_map(data, towns, "One-Year % Change in Price", legend="OneYear %Change" )

if export_html:
    fig.write_html("./html/boston-housing-1yc.html")
if export_figure:
    fig.write_image("./figures/boston-housing-1yc.png", scale=5)

In [ ]:
fig = plot_map(data, towns, "Five-Year % Change in Price" )

if export_html:
    fig.write_html("./html/boston-housing-5yc.html")

In [ ]:
fig = plot_map(data, towns, "Ten-Year % Change in Price", legend="TenYear %Change" )

if export_html:
    fig.write_html("./html/boston-housing-10yc.html")
if export_figure:
    fig.write_image("./figures/boston-housing-10yc.png", scale=5)